In [1]:
# !pip --default -timeout=100 install kafka-python
# !pip install pyhdfs
import numpy as np
import pandas as pd
import time
from datetime import datetime, timedelta
import json
import pyhdfs

In [ ]:
# #读取HDFS里的数据
# from pyhdfs import HdfsClient
# client=HdfsClient(hosts='10.100.100.100:9870') #hdfs地址
# res= client.open('/MODEL_DL_KZS5_CY/MODEL_DL_KZS5_CY20-06-29-14-40-00.1593412801584.log') #hdfs文件路径,根目录/
# df= pd.read_csv(res) #参数为源文件,编码,分隔符
# # for r in res:
# #     line=str(r)#open后是二进制,str()转换为字符串并转码
# #     print(line)

In [9]:
#消费kafka中最新的log文件数据
from kafka import KafkaConsumer

consumer = KafkaConsumer('MODEL_DL_KZS5_CY_KEP',bootstrap_servers=['10.100.100.105'],
                         auto_offset_reset='latest',
                         enable_auto_commit=True, # 自动提交消费数据的offset
                         consumer_timeout_ms= 10000, # 如果1秒内kafka中没有可供消费的数据，自动退出
                         value_deserializer=lambda m: json.loads(m.decode('ascii')),
                         client_id='consumer-python3'
                         )
res = consumer.poll(10)
# consumer.seek_to_end()

for message in consumer:
    most_recent_message_value = message.value
    break
#     #注意: message ,value都是原始的字节数据，需要decode
#     #例如: message.value.decode('utf-8')
#     [#完成对每条数据中的操作]
# #     print ("%s:%d:%d: key=%s value=%s" %s (message.topic, message.partition,
# #                                                message.offset, message.key,
# #                                                message.value))
#     print(message.topic, message.partition, message.offset, message.value)
#     print(message.topic, message.partition, message.offset, message.value)

In [10]:
# Change the datetime format to the timestamps
df = pd.DataFrame(index=['id', 'v', 'q', 't'])

for i in range(len(most_recent_message_value['values'])):
    df_temp = pd.DataFrame(most_recent_message_value['values'][i].items()).set_index(keys=0)
    df = pd.concat([df_temp, df], axis=1, join='inner')
    
df.columns = df.loc['id'].values
df = df.drop(df.index[0]).T

df = df[~df['v'].isin(['True','False'])]

df['v'] = df['v'].astype(float, inplace=True)
df['q'] = df['q'].astype(str, inplace=True)
df['t'] = df['t'].astype('int64', inplace=True)


df['t'] = df['t']*0.001

timestamp=[]
for i in df['t']:
    tmObject = time.localtime(i)
    timestamp.append(pd.Timestamp(time.strftime("%Y-%m-%d %X", tmObject)))
    
df['time'] = timestamp
df = df.drop(['t'], axis=1)

In [11]:
df

,v,q,time
DL.DL.DL_XZS5_SC_TAT7305_T1,30.202549,True,2020-07-03 10:19:20
DL.DL.DL_XZS5_SC_TAT7305_H1,45.580151,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_T3,30.086807,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_T2,29.508102,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_T1,28.640045,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_H3,49.363426,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_H2,49.739586,True,2020-07-03 10:19:20
DL.DL.DL_XZS4_SC_TAT7304_H1,52.546299,True,2020-07-03 10:19:20
DL.DL.DL_XZS3_SC_TAT7303_T3,28.379631,True,2020-07-03 10:19:20
DL.DL.DL_XZS3_SC_TAT7303_T2,30.376160,True,2020-07-03 10:19:20


In [12]:
kepware_plc_pivot = pd.read_csv('data/dianwei/kepware_plc_pivot_20200701.csv')

In [ ]:
# 导入点位pivot表,选取需要的点位，并配对选区的点位
kepware_plc_pivot = pd.read_csv('data/dianwei/kepware_plc_pivot_20200701.csv')
Kzs5_plc_pivot = kepware_plc_pivot[kepware_plc_pivot['设备编码'] =]
df_dianwei_select_pivot = kepware_plc_pivot[df_dianwei['kepware_plc_pivot'].isin(['测点5湿度',
'测点5温度',
'测点4湿度',
'测点4温度',
'测点3湿度',
'测点3温度',
'测点2湿度',
'测点2温度',
'测点1湿度',
'测点1温度',
'测点1温度',
'测点1湿度',
'回风湿度',
'回风温度',
'送风湿度',
'送风温度',
'新风湿度',
'新风温度',
'室外湿度',
'室外温度',
'新风阀开度反馈',
'混风阀开度反馈',
'表冷阀门开度反馈',
'加湿阀开度反馈',
'加热阀门开度反馈'
])]

df_dianwei_select = df_dianwei_select.set_index('点位')
df_dianwei_final = pd.concat([df, df_dianwei_select], axis=1, join='inner')

In [ ]:
df_dianwei_final

In [ ]:
df_dianwei_TH = df_dianwei_final[df_dianwei_final.index.isin(['DL_KZS5_CY_TAT7205-H5','DL_KZS5_CY_TAT7205-T5',
       'DL_KZS5_CY_TAT7205-H4', 'DL_KZS5_CY_TAT7205-T4',
       'DL_KZS5_CY_TAT7205-H3', 'DL_KZS5_CY_TAT7205-T3',
       'DL_KZS5_CY_TAT7205-H2', 'DL_KZS5_CY_TAT7205-T2',
       'DL_KZS5_CY_TAT7205-H1', 'DL_KZS5_CY_TAT7205-T1'])].T

In [ ]:
df_input = df_dianwei_final.T
df_input.columns= df_dianwei_final['点位名称']


df_input['Avg_H']=''
df_input['Avg_T']=''
df_input['时间']=''

if df_dianwei_final.q.all() == 'true':
    df_input.loc['v']['Avg_H'] = df_dianwei_TH.loc['v'].iloc[[0,2,4,6,8]].mean()
    df_input.loc['v']['Avg_T'] = df_dianwei_TH.loc['v'].iloc[[1,3,5,7,9]].mean()
    df_input['时间'] = df_dianwei_TH.loc['time'].mode()[0]

df_input.drop(['q','time','点位名称'], inplace= True)
df_input.drop(df_dianwei_TH.loc['点位名称'], axis=1, inplace= True)
df_input.reset_index(drop=True)
df_input.set_index('时间',inplace=True)

In [ ]:
df_input

In [ ]:
# rom kafka import KafkaConsumer

# consumer = KafkaConsumer('MODEL_DL_KZS5_CY',bootstrap_servers=['10.100.100.105'],
#                          auto_offset_reset='latest',
#                          enable_auto_commit=True, # 自动提交消费数据的offset
#                          consumer_timeout_ms= 10000, 2# 如果1秒内kafka中没有可供消费的数据，自动退出
#                          value_deserializer=lambda m: json.loads(m.decode('ascii')),
#                          client_id='consumer-python3'
#                          )
# res = consumer.poll(10)
# consumer.seek_to_end()

# for message in consumer:
#     most_recent_message_value = message.value
#     break
# #     #注意: message ,value都是原始的字节数据，需要decode

In [ ]:
dianwei = pd.DataFrame(df.columns.values)
dianwei.to_csv('dianwei.csv')

In [ ]:
# print(message.topic, message.partition, message.offset, message.value)

In [ ]:
# i = 0
# for i in range(600):  #模拟6w数据
#     while i < 100:  #每秒100条
#         # msg = '{"id":"K2_Z5.K2_Z5.K2_Z5GRP1DIAGSTART2","v":-1.783128E+22,"q":true,"t":623}'
#         msg= random.choice(data), datetime.datetime.now() #随机数据
#         print(msg)
#         producer.send('iot-kafka', value=msg[0].encode('utf-8')) #发送到laoshan-kafka
#         i += 1
#         # time.sleep(1)\

#     time.sleep(1)